## brand

---

In [ ]:
import os
import pandas as pd
import requests
import json
import warnings
import sys
sys.path.append('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data_collection/notiAPI')

from auth import NotiServerAuth
from api import NotiServerRequest

from dataclasses import dataclass, field
from typing import Optional, List
from urllib.parse import urlparse, parse_qs

from pprint import pprint

warnings.filterwarnings(action='ignore')

In [ ]:
# 최근 10일치 piv 데이터

df = pd.read_csv('/Users/daeheehan/final/level2-3-recsys-finalproject-recsys-02/data/df_interaction_unique.csv') 
len(df)

In [ ]:
df_test = df.head(20)

In [ ]:
# 주어진 데이터프레임에서 아이템 ID를 추출
item_ids = df_test['item'].tolist()

# 각 아이템에 대한 정보를 저장할 빈 리스트를 생성합니다.
brands = []

# 각 아이템에 대한 정보를 조회합니다.
# 브랜드 정보 조회후 브랜드 데이터가 조회되지 않는다면, title에서 brand를 가져옴
for item_id in item_ids:
    res = NotiServerRequest.product_info(ids=[], pivs=[item_id])
    try :
        brand = res.json().get('data', {}).get('products', [])[0]['details']['brand']
    except KeyError:
        try:
            brand = res.json().get('data', {}).get('products', [])[0]['title'].split()[0]
        except :
            brand = 'unknown'
    except:
        brand = 'unknown'

    brands.append(brand)

# 브랜드 정보를 데이터프레임에 새로운 열로 추가합니다.
df_test['brand'] = brands
df_test

---

## cate

- 키워드 기반 분류
    - 특정 키워드나 패턴이 포함된 제품명을 해당 카테고리에 할당합니다. 
    - 예를 들어, "TV"나 "UHD"가 포함되어 있으면 "TV" 카테고리로, "노트북"이나 "코어"가 포함되어 있으면 "노트북" 카테고리로 분류할 수 있습니다.
    - 키워드 리스트 생성 : 제품명에서 빈도수가 많이 나오는 애들 힌트를 얻어서 처리


In [ ]:
cate = [
    "노트북",     # laptop
    "태블릿",     # tablet
    "스마트폰",   # smartphone
    "스마트워치", # smartwatch
    "모니터",     # monitor
    "TV",         # tv
    "스피커",     # speaker
    "헤드폰",     # headphone
    "이어폰",     # earphone
    "게이밍",      # gaming
    "데스크탑",    # desktop
    "키보드",     # keyboard
    "마우스",     # mouse
    "로봇청소기",  # robovac
    "냉장고",     # fridge
    "세탁기",     # laundry
    "킥보드",     # scooter
    "저장장치",   # storage
    "청소기",     # cleaner
    "가전/디지털" # digital
]


In [ ]:
# 주어진 데이터프레임에서 아이템 ID를 추출
item_ids = df_test['item'].tolist()

# 각 아이템에 대한 정보를 저장할 빈 리스트를 생성합니다.
cates = []

# 각 아이템에 대한 정보를 조회합니다.
for item_id in item_ids:
    try:
        res = NotiServerRequest.product_info(ids=[], pivs=[item_id])
        # 'details'에 'brand' 정보가 있는지 확인하고, 없는 경우 'title' 정보로 대체합니다.
        cate = res.json().get('data', {}).get('products', [])[0]['details']['category_small']
    except KeyError:
        try :
            cate = res.json().get('data', {}).get('products', [])[0]['title'].split(",")[0]
        except :
            cate = 'unknown'
    except :
        cate = 'unknown'
    
    cates.append(cate)

        
# 브랜드 정보를 데이터프레임에 새로운 열로 추가합니다.
df_test['cate'] = cates
df_test

In [ ]:
# 카테고리별 키워드 정의
category_keywords = {
    '노트북': ['노트북','맥북', '그램','ThinkPad','FLEX','스트릭스','스워드','Zenbook','오멘','블레이드','에일리언웨어', '갤럭시북', '아이디어패드','울트라PC','코어i7', '코어i5', '코어i9','라이젠'],
    '스마트폰': ['스마트폰'],
    '헤드폰': ['헤드폰','헤드셋'],
    '스피커': ['스피커'],
    '모니터': ['모니터'],
    '냉장고': ['냉장고'],
    '마우스': ['마우스', 'G PRO'],
    '청소기': ['청소기'],
    '키보드': ['키보드','키크론'],
    'TV': ['TV','HD'],
    '세탁기': ['세탁기'],         
    '가전/디지털': ['가전/디지털'],  
    '로봇청소기': ['로봇청소기'],
    '게이밍': ['게이밍'],
    '이어폰': ['이어폰'],
    '저장장치': ['저장장치'],
    'unknown' : ['unknown']
}

# 분류를 위한 함수 정의
def categorize_by_keywords(item, category_keywords):
    for category, keywords in category_keywords.items():
        if any(keyword in item for keyword in keywords):
            return category
    return item  # 키워드가 일치하지 않는 경우 etc 처리

# 'cate' 열에 함수 적용
df_test['cate'] = df_test['cate'].apply(lambda x: categorize_by_keywords(x, category_keywords))
df_test

---